<h2 style="text-align: center;">AskAdam🏀💻🔍</h1>

#### Install and Import Necessary Packages

In [1]:
# !pip install openai
# !pip install openai==1.63.2
# !pip install --upgrade openai
# !python -m pip install --upgrade pip
# !python -m pip install --force-reinstall openai

# !pip install openai==0.28.0
# !pip install pyarrow
# !pip show openai
# !pip show pandas
# !pip show dotenv

In [2]:
# !pip install streamlit
# !pip install tabulate

In [1]:
import os
import time
import pandas as pd
import numpy as np
import faiss
import openai
from dotenv import load_dotenv
from openai.error import RateLimitError
from pandasql import sqldf
from IPython.display import display, HTML

#### Configure Azure OpenAI (adjust these values in your .env)

In [2]:
openai.api_type = "azure"
openai.api_base = "https://your-resource-name.openai.azure.com/"
openai.api_version = "2022-12-01"  
openai.api_key = "your_api_key"
AZURE_OPENAI_EMBEDDING_DEPLOYMENT="your_embedding_deployment"
AZURE_OPENAI_CHAT_DEPLOYMENT = "your_embedding_deployment"

#### Load Precomputed Dataset from Parquet, Convert Embeddings to NumPy arrays, and Select Features

In [3]:
def load_vectorized_data(parquet_path='master_agent_data_with_embeddings.parquet'):
    """Load the precomputed dataset from Parquet and convert embeddings to NumPy arrays."""
    df = pd.read_parquet(parquet_path)
    return df

In [4]:
df = load_vectorized_data()

In [5]:
df_new = df[[ "TEAM", "CONF", "DIVISION", "GP", "PPG", "oPPG", "pDIFF", "PACE", "oEFF", "dEFF", "W", "L", "WIN%", "SEASON", "embedding"]]

In [6]:
# df_new.columns = ['TEAM', 'CONF', 'DIVISION', 'GP', 'PPG', 'SEASON', 'embedding']

#### Function to Load OpenAI Model and Get SQL Query as a Response

In [7]:
def get_openai_response(question, prompt):
    messages = [
        {"role": "system", "content": prompt[0]},
        {"role": "user", "content": question}
    ]
    response = openai.ChatCompletion.create(
        engine=AZURE_OPENAI_CHAT_DEPLOYMENT,  # Use your Azure chat deployment name
        messages=messages,
        temperature=0.2
    )
    return response['choices'][0]['message']['content'].strip()

#### Function to Retrieve Query from The Pandas DataFrame Using SQL

In [8]:
def run_sql_query(sql, df_new):
    return sqldf(sql, locals())

#### Define Prompt for Generating SQL Query

In [9]:
prompt = [
    """
    Sir, you are an expert in converting English questions to SQL query!
    The SQL table is named 'df_new' and has the following columns -  "TEAM", "CONF", "DIVISION", "GP", "PPG", "oPPG", "pDIFF", "PACE", "oEFF", "dEFF", 
    "W", "L", "WIN%", "SEASON", "embedding"
    
    Also, here are unique values in some variables for reference and applying correct values in filters:
    SEASON : ["2017-2018", "2018-2019", "2019-2020", "2021-2022", "2022-2023", "2023-2024", "2024-2025"]
    CONF : ["East", "West"]
    DIVISION : ["Atlantic", "Central", "Northwest", "Pacific", "Southeast", "Southwest"]
    TEAM : ["Atlanta", "Boston", "Brooklyn", "Charlotte", "Chicago", "Cleveland", "Dallas", "Denver", "Detroit", 
         "Golden State", "Houston", "Indiana", "LA Clippers", "LA Lakers", "Memphis", "Miami", "Milwaukee", 
         "Minnesota", "New Orleans", "New York", "Oklahoma City", "Orlando", "Philadelphia", "Phoenix", 
         "Portland", "Sacramento", "San Antonio", "Toronto", "Utah", "Washington"]
    
    Feature "dEFF" is an inverse feature, meaning the higher the value the worse it is. So, the row with the highest value in that feautre would be worst,
    the row with the lowest value would be best.
    
    For example,
    Example 1 - How many entries of records are present? 
    The SQL command will be: SELECT COUNT(*) as No_of_Records FROM df_new;

    Example 2 - Which five teams led the league in scoring during the 2022-2023 season?
    The SQL command will be: SELECT TEAM, PPG FROM df_new WHERE SEASON = '2022-2023' ORDER BY PPG DESC LIMIT 5;
    GROUP BY Race, Rural_Urban_Flag;

    Example 3 - How mnany games did the LA Lakers play in 24/25?
    The SQL command will be: SELECT GP FROM df_new WHERE team = 'LA Lakers' AND season = '2024-2025';

    Example 4 - What was the LA Lakers record in 22/23?
    The SQL command will be: SELECT TEAM, CONCAT(W, '-', L) AS RECORD FROM df_new WHERE team = 'LA Lakers' AND season = '2022-2023';

    Make sure record is a combination of features W and L - as explained in Example 4 above.
    Make sure the output is only the SQL query, without any explanation or markdown formatting.
    Make sure it ignores the case of variable to filter in the select query.
    If multiple variables are mentioned, please provide a group by query.
    Please treat SEASON and Year as same keywords.

    """
]


In [24]:
# Define your prompt for final outcome
prompt_instruction = [
    """
        Sir, you are a helpful assistant who turns data tables and questions into friendly and concise summaries.
        
        Add line breaks or '/n' wherever you feel should be there to make the response more readable. 
                        
    """
]

#### Define Questions to be Answered

In [21]:
# question = "What team led the leage in defensive efficiency during the 24/25 season, what team ranked worst in defensive efficiency that same year, and what was the difference between the two?"
# question2 = "What team led the leage in defensive efficiency during the 24/25 season, what team ranked worst in defensive efficiency that same year, and what was the difference between the two?"
# question3 = "What was the Knicks record in 2022-2023 compared to their record in 2023-2024? What was the differenfce in games won?"

In [22]:
sql_query = get_openai_response(question, prompt)
print(sql_query)

SELECT TEAM, SEASON, CONCAT(W, '-', L) AS RECORD, W AS GAMES_WON FROM df_new WHERE TEAM = 'New York' AND SEASON IN ('2022-2023', '2023-2024') GROUP BY TEAM, SEASON;


In [23]:
result_df = run_sql_query(sql_query, df_new)

#### Function to Generate LLM Response

In [25]:
def generate_natural_language_summary(question, result_df, prompt_instruction):
    table_text = result_df.to_markdown(index=False)
 
    # Build system and user messages
    messages = [
        {"role": "system", "content": prompt_instruction},
        {"role": "user", "content": f"User question: {question}\n\nHere is the data:\n{table_text}\n\nPlease summarize this in clear, human-readable English."}
    ]
    
    # Make API call
    response = openai.ChatCompletion.create(
        engine=AZURE_OPENAI_CHAT_DEPLOYMENT,  # Use your Azure chat deployment name
        messages=messages,
        temperature=0.3,
        max_tokens=430
    )
    
    return response['choices'][0]['message']['content'].strip()


#### Print Questions and LLM Responses

In [43]:
response_for_ui = generate_natural_language_summary(question, result_df, prompt_instruction[0])
display(HTML(f"<b>Question:</b> {question}"))
display(HTML(f"<b>AskAdam Response:</b> {response_for_ui}"))

In [16]:
response_for_ui = generate_natural_language_summary(question, result_df, prompt_instruction[0])
display(HTML(f"<b>Question:</b> {question}"))
display(HTML(f"<b>AskAdam Response:</b> {response_for_ui}"))

In [26]:
response_for_ui = generate_natural_language_summary(question, result_df, prompt_instruction[0])
display(HTML(f"<b>Question:</b> {question}"))
display(HTML(f"<b>AskAdam Response:</b> {response_for_ui}"))